# `--- Day 4: Passport Processing ---`
```
You arrive at the airport only to realize that you grabbed your North Pole Credentials instead of your passport. While these documents are extremely similar, North Pole Credentials aren't issued by a country and therefore aren't actually valid documentation for travel in most of the world.

It seems like you're not the only one having problems, though; a very long line has formed for the automatic passport scanners, and the delay could upset your travel itinerary.

Due to some questionable network security, you realize you might be able to solve both of these problems at the same time.

The automatic passport scanners are slow because they're having trouble detecting which passports have all required fields. The expected fields are as follows:

    byr (Birth Year)
    iyr (Issue Year)
    eyr (Expiration Year)
    hgt (Height)
    hcl (Hair Color)
    ecl (Eye Color)
    pid (Passport ID)
    cid (Country ID)

Passport data is validated in batch files (your puzzle input). Each passport is represented as a sequence of key:value pairs separated by spaces or newlines. Passports are separated by blank lines.

Here is an example batch file containing four passports:

ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in

The first passport is valid - all eight fields are present. The second passport is invalid - it is missing hgt (the Height field).

The third passport is interesting; the only missing field is cid, so it looks like data from North Pole Credentials, not a passport at all! Surely, nobody would mind if you made the system temporarily ignore missing cid fields. Treat this "passport" as valid.

The fourth passport is missing two fields, cid and byr. Missing cid is fine, but missing any other field is not, so this passport is invalid.

According to the above rules, your improved system would report 2 valid passports.

Count the number of valid passports - those that have all required fields. Treat cid as optional. In your batch file, how many passports are valid?
```

## `--- Part Two ---`
```
The line is moving more quickly now, but you overhear airport security talking about how passports with invalid data are getting through. Better add some data validation, quick!

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

    byr (Birth Year) - four digits; at least 1920 and at most 2002.
    iyr (Issue Year) - four digits; at least 2010 and at most 2020.
    eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
    hgt (Height) - a number followed by either cm or in:
        If cm, the number must be at least 150 and at most 193.
        If in, the number must be at least 59 and at most 76.
    hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
    ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    pid (Passport ID) - a nine-digit number, including leading zeroes.
    cid (Country ID) - ignored, missing or not.

Your job is to count the passports where all required fields are both present and valid according to the above rules. Here are some example values:

byr valid:   2002
byr invalid: 2003

hgt valid:   60in
hgt valid:   190cm
hgt invalid: 190in
hgt invalid: 190

hcl valid:   #123abc
hcl invalid: #123abz
hcl invalid: 123abc

ecl valid:   brn
ecl invalid: wat

pid valid:   000000001
pid invalid: 0123456789

Here are some invalid passports:

eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007

Here are some valid passports:

pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719

Count the number of valid passports - those that have all required fields and valid values. Continue to treat cid as optional. In your batch file, how many passports are valid?
```

In [1]:
import pandas as pd
import numpy as np

import functools
import operator

In [2]:
test_txt = '''ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
'''

In [3]:
with open('input.txt') as fd:
    real_txt = fd.read()

In [4]:
def parse_input(text):
    data = list()
    passport = dict()

    for line in text.splitlines():
        #print(f'line={line}')
        if len(line) == 0:
            data.append(passport.copy())
            passport = dict()
            continue
        for kv in line.split(' '):
            #print(f'kv = {kv}')
            k, v = kv.split(':')
            passport[k] = v
    if len(passport) != 0:
        data.append(passport.copy())
    return pd.DataFrame(data)
            

## Part 1

In [5]:
required_fields = [
    'byr',
    'iyr',
    'eyr',
    'hgt',
    'hcl',
    'ecl',
    'pid',
#     'cid',
]

In [6]:
for desc, input_txt in [
    ('test_batch (should == 2)', test_txt),
    ('>>> Part 1 Answer', real_txt)
]:
    password_db = parse_input(input_txt).dropna(subset=required_fields, how='any')

    valid_passports = password_db.shape[0]
    print(f'{desc:>24s}: ', valid_passports)

test_batch (should == 2):  2
       >>> Part 1 Answer:  192


## Part 2
### Validation rules

```
    byr (Birth Year) - four digits; at least 1920 and at most 2002.
    iyr (Issue Year) - four digits; at least 2010 and at most 2020.
    eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
    hgt (Height) - a number followed by either cm or in:
        If cm, the number must be at least 150 and at most 193.
        If in, the number must be at least 59 and at most 76.
    hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
    ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    pid (Passport ID) - a nine-digit number, including leading zeroes.
    cid (Country ID) - ignored, missing or not.
```

In [7]:
p2_invalid = '''eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
'''

p2_valid = '''pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
'''

In [8]:
for desc, input_txt in [
    ('valid_passport_test (should == 4)', p2_valid), 
    ('invalid_passport_test (should == 0)', p2_invalid),
    ('test_batch (should == 2)', test_txt),
    ('>>> Part 2 Answer', real_txt)
]:
    password_db = parse_input(input_txt).dropna(subset=required_fields, how='any')

    f_byr = (password_db['byr'].astype(np.int) >= 1920) & (password_db['byr'].astype(np.int) <= 2002)

    f_iyr = (password_db['iyr'].astype(np.int) >= 2010) & (password_db['iyr'].astype(np.int) <= 2020)

    f_eyr = (password_db['eyr'].astype(np.int) >= 2020) & (password_db['eyr'].astype(np.int) <= 2030)

    hgt_int = password_db['hgt'].str.replace('in', '').str.replace('cm', '').astype(np.int)
    hgt_int.name='hgt_int'
    f_hgt = (password_db['hgt'].str.endswith('in') & (hgt_int >= 59) & (hgt_int <= 76)) | (password_db['hgt'].str.endswith('cm') & (hgt_int >= 150) & (hgt_int <= 193))

    f_hcl = (password_db['hcl'].str.match(r'^#[0-9a-f]{6}$'))

    f_ecl = (password_db['ecl'].isin(['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']))

    f_pid = (password_db['pid'].str.match(r'^[0-9]{9}$'))
    
    print(f'{desc:>36s}: ', password_db.loc[
        functools.reduce(operator.and_, [f_byr, f_iyr, f_eyr, f_hgt, f_hcl, f_ecl, f_pid])
    ].shape[0])

   valid_passport_test (should == 4):  4
 invalid_passport_test (should == 0):  0
            test_batch (should == 2):  2
                   >>> Part 2 Answer:  101


### Explore valid/invalid entries with dataframe styling

In [9]:
password_db = parse_input(real_txt).dropna(subset=required_fields, how='any')

In [10]:
valid = functools.reduce(operator.and_, [f_byr, f_iyr, f_eyr, f_hgt, f_hcl, f_ecl, f_pid])
valid.name = 'valid'

In [11]:
def highlight_invalid(df):
    attr = 'background-color: yellow'
    styles = pd.DataFrame(columns=df.columns, index=df.index).fillna(False)
    styles['byr'] = ~f_byr[df.index]
    styles['iyr'] = ~f_iyr[df.index]
    styles['eyr'] = ~f_eyr[df.index]
    styles['ecl'] = ~f_ecl[df.index]
    styles['pid'] = ~f_pid[df.index]
    styles['hgt'] = ~f_hgt[df.index]
    styles['hcl'] = ~f_hcl[df.index]
    return pd.DataFrame(np.where(styles, attr, ''), columns=df.columns, index=df.index)

In [12]:
password_db.merge(hgt_int, left_index=True, right_index=True).merge(valid, left_index=True, right_index=True).sample(10).style.apply(highlight_invalid, axis=None)

,byr,ecl,pid,eyr,iyr,hcl,hgt,cid,hgt_int,valid
188,1937,brn,990171473,2028,2015,#fffffd,165cm,68,165,True
75,2009,grt,996363336,1946,2000,7be409,72cm,292,72,False
158,1997,#faa530,923359071,2028,2013,e6c902,177cm,nan,177,False
58,1960,blu,752655640,2028,2012,#888785,191cm,249,191,True
64,1981,blu,216910202,2025,2013,#866857,157cm,nan,157,True
177,1954,amb,135511825,2020,2017,#c0946f,68in,nan,68,True
253,1993,brn,855375268,2028,2018,#341e13,179cm,nan,179,True
230,1921,oth,021318713,2021,2018,#b6652a,168cm,nan,168,True
146,1971,hzl,458416257,2023,2019,#6b5442,183cm,345,183,True
242,1923,hzl,321400085,2023,2016,#733820,189cm,nan,189,True
